# Salary Predictions Based on Job Descriptions

# Part 1 - DEFINE

### ---- 1 Define the problem ----

Write the problem in your own words here

In [74]:
#import your libraries
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
#etc

#your info here
__author__ = "Kartik Athale "
__email__ = "Kartik.athale@outlook.com"

## Part 2 - DISCOVER

### ---- 2 Load the data ----

In [2]:
#load the data into a Pandas dataframe

# Path of the file to read.
#Sal_pred_path = 'C:\\Users\\Kartik Athale\\Desktop\\Big Data Program\\DSDJ\\Portfolio\\salarypredictionportfolio\\data'

#train_df = pd.read_csv('C:\\Users\\Kartik Athale\\Desktop\\Big Data Program\\DSDJ\\Portfolio\\salarypredictionportfolio\\data\\train_salaries.csv')

train_sal_df = pd.read_csv('data/train_salaries.csv')
train_feature_df = pd.read_csv('data/train_features.csv')
test_feature_df = pd.read_csv('data/test_features.csv')


### Observing Data ..

In [3]:
train_sal_df.head()

,jobId,salary
0,JOB1362684407687,130
1,JOB1362684407688,101
2,JOB1362684407689,137
3,JOB1362684407690,142
4,JOB1362684407691,163


In [4]:
train_feature_df.head()

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
0,JOB1362684407687,COMP37,CFO,MASTERS,MATH,HEALTH,10,83
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73
2,JOB1362684407689,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38
3,JOB1362684407690,COMP38,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17
4,JOB1362684407691,COMP7,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16


In [9]:
test_feature_df.head()

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
0,JOB1362685407687,COMP33,MANAGER,HIGH_SCHOOL,NONE,HEALTH,22,73
1,JOB1362685407688,COMP13,JUNIOR,NONE,NONE,AUTO,20,47
2,JOB1362685407689,COMP10,CTO,MASTERS,BIOLOGY,HEALTH,17,9
3,JOB1362685407690,COMP21,MANAGER,HIGH_SCHOOL,NONE,OIL,14,96
4,JOB1362685407691,COMP36,JUNIOR,DOCTORAL,BIOLOGY,OIL,10,44


###  Data types and dimention..

In [13]:
print('Target Salary Dimension', train_sal_df.shape)
print('train_feature Dimension',train_feature_df.shape)
print('test_feature_df Dimension',test_feature_df.shape)

Target Salary Dimension (1000000, 2)
train_feature Dimension (1000000, 8)
test_feature_df Dimension (1000000, 8)


In [15]:
train_sal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
jobId     1000000 non-null object
salary    1000000 non-null int64
dtypes: int64(1), object(1)
memory usage: 15.3+ MB


In [6]:
train_feature_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
jobId                  1000000 non-null object
companyId              1000000 non-null object
jobType                1000000 non-null object
degree                 1000000 non-null object
major                  1000000 non-null object
industry               1000000 non-null object
yearsExperience        1000000 non-null int64
milesFromMetropolis    1000000 non-null int64
dtypes: int64(2), object(6)
memory usage: 61.0+ MB


In [18]:
test_feature_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
jobId                  1000000 non-null object
companyId              1000000 non-null object
jobType                1000000 non-null object
degree                 1000000 non-null object
major                  1000000 non-null object
industry               1000000 non-null object
yearsExperience        1000000 non-null int64
milesFromMetropolis    1000000 non-null int64
dtypes: int64(2), object(6)
memory usage: 61.0+ MB


In [5]:
## Summary of the Train data
train_feature_df.describe()

,yearsExperience,milesFromMetropolis
count,1000000.000000,1000000.000000
mean,11.992386,49.529260
std,7.212391,28.877733
min,0.000000,0.000000
25%,6.000000,25.000000
50%,12.000000,50.000000
75%,18.000000,75.000000
max,24.000000,99.000000


In [17]:
train_sal_df.describe()

,salary
count,1000000.000000
mean,116.061818
std,38.717936
min,0.000000
25%,88.000000
50%,114.000000
75%,141.000000
max,301.000000


### ---- 3 Clean the data ----

In [3]:
#look for duplicate data, invalid data (e.g. salaries <=0), or corrupt data and remove it
# Checking for NAs in the Dataset.
print("---Salary dataset---")
print(train_sal_df.isna().sum())
print()
print("---Train dataset---")
print(train_feature_df.isna().sum())
print()
print("---Test dataset---")
print(test_feature_df.isna().sum())

---Salary dataset---
jobId     0
salary    0
dtype: int64

---Train dataset---
jobId                  0
companyId              0
jobType                0
degree                 0
major                  0
industry               0
yearsExperience        0
milesFromMetropolis    0
dtype: int64

---Test dataset---
jobId                  0
companyId              0
jobType                0
degree                 0
major                  0
industry               0
yearsExperience        0
milesFromMetropolis    0
dtype: int64


In [29]:
train_sal_df.duplicated().sum()

0

In [31]:
train_feature_df.duplicated().sum()

0

In [32]:
test_feature_df.duplicated().sum()

0

In [34]:
# Training dataset Column
train_feature_df.columns

Index(['jobId', 'companyId', 'jobType', 'degree', 'major', 'industry',
       'yearsExperience', 'milesFromMetropolis'],
      dtype='object')

In [7]:
numeric_cols = ['yearsExperience', 'milesFromMetropolis']

In [8]:
categorical_cols = ['jobId', 'companyId', 'jobType', 'degree', 'major', 'industry']

In [9]:
# Merge the features and salaries on jobId, 
train_df = pd.merge(train_feature_df, train_sal_df, on='jobId')

In [10]:
del train_feature_df
del train_sal_df

In [37]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
jobId                  1000000 non-null object
companyId              1000000 non-null object
jobType                1000000 non-null object
degree                 1000000 non-null object
major                  1000000 non-null object
industry               1000000 non-null object
yearsExperience        1000000 non-null int64
milesFromMetropolis    1000000 non-null int64
salary                 1000000 non-null int64
dtypes: int64(3), object(6)
memory usage: 76.3+ MB


In [11]:
train_df.head()

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary
0,JOB1362684407687,COMP37,CFO,MASTERS,MATH,HEALTH,10,83,130
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73,101
2,JOB1362684407689,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38,137
3,JOB1362684407690,COMP38,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17,142
4,JOB1362684407691,COMP7,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16,163


In [41]:
#Checking for Invalid data, if Salary < 0
train_df[train_df.salary < 0]

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary


### ---- 4 Explore the data (EDA) ----

In [3]:
#summarize each feature variable
#summarize the target variable
#look for correlation between each feature and the target
#look for correlation between features

In [42]:
train_df.describe()

,yearsExperience,milesFromMetropolis,salary
count,1000000.000000,1000000.000000,1000000.000000
mean,11.992386,49.529260,116.061818
std,7.212391,28.877733,38.717936
min,0.000000,0.000000,0.000000
25%,6.000000,25.000000,88.000000
50%,12.000000,50.000000,114.000000
75%,18.000000,75.000000,141.000000
max,24.000000,99.000000,301.000000


In [49]:
df_jobtype = train_df.groupby(['jobType'])
df_jobtype.mean()
#train_df.groupby(['jobType'])[['salary']].mean()

,yearsExperience,milesFromMetropolis,salary
jobType,,,
CEO,11.998269,49.478458,145.311425
CFO,11.990440,49.572715,135.458547
CTO,12.010860,49.580847,135.481067
JANITOR,11.969657,49.443815,70.813045
JUNIOR,11.992383,49.660508,95.333087
MANAGER,11.987284,49.498354,115.368518
SENIOR,11.996624,49.572558,105.487775
VICE_PRESIDENT,11.993700,49.428222,125.368630


In [40]:
train_df.loc[train_df.salary > 220.0, 'jobType'].value_counts()

CEO               3227
CFO               1496
CTO               1488
VICE_PRESIDENT     603
MANAGER            217
SENIOR              66
JUNIOR              20
Name: jobType, dtype: int64

In [47]:
train_df[train_df['salary'] < 10]

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary
30559,JOB1362684438246,COMP44,JUNIOR,DOCTORAL,MATH,AUTO,11,7,0
495984,JOB1362684903671,COMP34,JUNIOR,NONE,NONE,OIL,1,25,0
652076,JOB1362685059763,COMP25,CTO,HIGH_SCHOOL,NONE,AUTO,6,60,0
816129,JOB1362685223816,COMP42,MANAGER,DOCTORAL,ENGINEERING,FINANCE,18,6,0
828156,JOB1362685235843,COMP40,VICE_PRESIDENT,MASTERS,ENGINEERING,WEB,3,29,0


In [50]:
# Remove data with zero salaries
train_df = train_df[train_df.salary > 10]

In [53]:
train_df.groupby(['jobType'])[['salary']].mean()

,salary
jobType,
CEO,145.311425
CFO,135.458547
CTO,135.481067
JANITOR,70.813045
JUNIOR,95.333087
MANAGER,115.368518
SENIOR,105.487775
VICE_PRESIDENT,125.368630


In [65]:
categorical_vars = ['companyId', 'jobType', 'degree', 'major', 'industry']
numeric_vars = ['yearsExperience', 'milesFromMetropolis']
target_var = 'salary'

In [66]:
target_var

'salary'

In [67]:
train_df = pd.get_dummies(train_df, columns = categorical_vars)  

In [70]:
train_df.head()

,jobId,yearsExperience,milesFromMetropolis,salary,companyId_COMP0,companyId_COMP1,companyId_COMP10,companyId_COMP11,companyId_COMP12,companyId_COMP13,...,major_MATH,major_NONE,major_PHYSICS,industry_AUTO,industry_EDUCATION,industry_FINANCE,industry_HEALTH,industry_OIL,industry_SERVICE,industry_WEB
0,JOB1362684407687,10,83,130,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
1,JOB1362684407688,3,73,101,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2,JOB1362684407689,10,38,137,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,JOB1362684407690,8,17,142,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,JOB1362684407691,8,16,163,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0


In [71]:
X = train_df.drop(axis=1,
                  columns=['jobId', 'salary' ]
                  )

In [72]:
y = train_df['salary']

### ---- 5 Establish a baseline ----

In [5]:
#select a reasonable metric (MSE in this case)
#create an extremely simple model and measure its efficacy
#e.g. use "average salary" for each industry as your model and then measure MSE
#during 5-fold cross-validation

In [76]:
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size = 0.20,  random_state=1)


In [77]:
DT_Model = DecisionTreeRegressor(random_state=1)
# Fit Model
DT_Model.fit(train_X, train_y)


DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=1, splitter='best')

In [78]:
# get predicted Salary on validation data
val_predictions = DT_Model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

#mean_squared_error

20.588995028308478


In [80]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(val_y, val_predictions))


698.3414656309392


In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=150, n_jobs=2, max_depth=25, min_samples_split=60,
                                 max_features=30, verbose=0, random_state=1)

# fit your model
rf_model.fit(train_X, train_y)


rf_predict = rf_model.predict(val_X)
print(mean_squared_error(val_y, val_predictions))


### ---- 6 Hypothesize solution ----

In [ ]:
#brainstorm 3 models that you think may improve results over the baseline model based
#on your 

Brainstorm 3 models that you think may improve results over the baseline model based on your EDA and explain why they're reasonable solutions here.

Also write down any new features that you think you should try adding to the model based on your EDA, e.g. interaction variables, summary statistics for each group, etc

## Part 3 - DEVELOP

You will cycle through creating features, tuning models, and training/validing models (steps 7-9) until you've reached your efficacy goal

#### Your metric will be MSE and your goal is:
 - <360 for entry-level data science roles
 - <320 for senior data science roles

### ---- 7 Engineer features  ----

In [ ]:
#make sure that data is ready for modeling
#create any new features needed to potentially enhance model

### ---- 8 Create models ----

In [15]:
#create and tune the models that you brainstormed during part 2

### ---- 9 Test models ----

In [1]:
#do 5-fold cross validation on models and measure MSE

### ---- 10 Select best model  ----

In [ ]:
#select the model with the lowest error as your "prodcuction" model

## Part 4 - DEPLOY

### ---- 11 Automate pipeline ----

In [ ]:
#write script that trains model on entire training set, saves model to disk,
#and scores the "test" dataset

### ---- 12 Deploy solution ----

In [16]:
#save your prediction to a csv file or optionally save them as a table in a SQL database
#additionally, you want to save a visualization and summary of your prediction and feature importances
#these visualizations and summaries will be extremely useful to business stakeholders

### ---- 13 Measure efficacy ----

We'll skip this step since we don't have the outcomes for the test data